# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 4:58PM,253053,15,8896945,"Brookfield Pharmaceuticals, LLC",Released
1,Dec 13 2022 4:58PM,253053,15,8896946,"Brookfield Pharmaceuticals, LLC",Released
2,Dec 13 2022 4:58PM,253053,15,8896947,"Brookfield Pharmaceuticals, LLC",Released
3,Dec 13 2022 4:58PM,253053,15,8896948,"Brookfield Pharmaceuticals, LLC",Released
4,Dec 13 2022 4:58PM,253053,15,8896949,"Brookfield Pharmaceuticals, LLC",Released
5,Dec 13 2022 4:58PM,253053,15,8896950,"Brookfield Pharmaceuticals, LLC",Released
6,Dec 13 2022 4:58PM,253053,15,8896951,"Brookfield Pharmaceuticals, LLC",Released
7,Dec 13 2022 4:58PM,253053,15,8896952,"Brookfield Pharmaceuticals, LLC",Released
8,Dec 13 2022 4:58PM,253053,15,8896953,"Brookfield Pharmaceuticals, LLC",Released
9,Dec 13 2022 4:58PM,253053,15,8896954,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,253041,Released,1
36,253049,Released,1
37,253051,Released,15
38,253052,Released,1
39,253053,Released,25


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253041,NaN,NaN,1.0
253049,NaN,NaN,1.0
253051,NaN,NaN,15.0
253052,NaN,NaN,1.0
253053,NaN,NaN,25.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252958,0.0,0.0,1.0
252961,0.0,0.0,8.0
252963,0.0,0.0,1.0
252964,0.0,0.0,1.0
252967,1.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252958,0,0,1
252961,0,0,8
252963,0,0,1
252964,0,0,1
252967,1,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252958,0,0,1
1,252961,0,0,8
2,252963,0,0,1
3,252964,0,0,1
4,252967,1,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252958,,,1
1,252961,,,8
2,252963,,,1
3,252964,,,1
4,252967,1,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 4:58PM,253053,15,"Brookfield Pharmaceuticals, LLC"
25,Dec 13 2022 4:57PM,253051,15,"Alliance Pharma, Inc."
40,Dec 13 2022 4:56PM,253052,10,"TherapeuticsMD, Inc."
41,Dec 13 2022 4:40PM,253049,15,Lonza
42,Dec 13 2022 4:16PM,253040,10,TherapeuticsMD Elli Sample
43,Dec 13 2022 4:09PM,253041,10,Emerginnova
44,Dec 13 2022 4:08PM,253039,10,ISDIN Corporation
48,Dec 13 2022 3:48PM,253035,16,Sartorius Bioprocess Solutions
49,Dec 13 2022 3:07PM,253027,10,"Snap Medical Industries, LLC"
51,Dec 13 2022 2:46PM,253025,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 4:58PM,253053,15,"Brookfield Pharmaceuticals, LLC",,,25
1,Dec 13 2022 4:57PM,253051,15,"Alliance Pharma, Inc.",,,15
2,Dec 13 2022 4:56PM,253052,10,"TherapeuticsMD, Inc.",,,1
3,Dec 13 2022 4:40PM,253049,15,Lonza,,,1
4,Dec 13 2022 4:16PM,253040,10,TherapeuticsMD Elli Sample,,1,
5,Dec 13 2022 4:09PM,253041,10,Emerginnova,,,1
6,Dec 13 2022 4:08PM,253039,10,ISDIN Corporation,,1,3
7,Dec 13 2022 3:48PM,253035,16,Sartorius Bioprocess Solutions,,,1
8,Dec 13 2022 3:07PM,253027,10,"Snap Medical Industries, LLC",,,2
9,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 4:58PM,253053,15,"Brookfield Pharmaceuticals, LLC",25,,
1,Dec 13 2022 4:57PM,253051,15,"Alliance Pharma, Inc.",15,,
2,Dec 13 2022 4:56PM,253052,10,"TherapeuticsMD, Inc.",1,,
3,Dec 13 2022 4:40PM,253049,15,Lonza,1,,
4,Dec 13 2022 4:16PM,253040,10,TherapeuticsMD Elli Sample,,1,
5,Dec 13 2022 4:09PM,253041,10,Emerginnova,1,,
6,Dec 13 2022 4:08PM,253039,10,ISDIN Corporation,3,1,
7,Dec 13 2022 3:48PM,253035,16,Sartorius Bioprocess Solutions,1,,
8,Dec 13 2022 3:07PM,253027,10,"Snap Medical Industries, LLC",2,,
9,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 4:58PM,253053,15,"Brookfield Pharmaceuticals, LLC",25,,
1,Dec 13 2022 4:57PM,253051,15,"Alliance Pharma, Inc.",15,,
2,Dec 13 2022 4:56PM,253052,10,"TherapeuticsMD, Inc.",1,,
3,Dec 13 2022 4:40PM,253049,15,Lonza,1,,
4,Dec 13 2022 4:16PM,253040,10,TherapeuticsMD Elli Sample,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 4:58PM,253053,15,"Brookfield Pharmaceuticals, LLC",25.0,NaN,NaN
1,Dec 13 2022 4:57PM,253051,15,"Alliance Pharma, Inc.",15.0,NaN,NaN
2,Dec 13 2022 4:56PM,253052,10,"TherapeuticsMD, Inc.",1.0,NaN,NaN
3,Dec 13 2022 4:40PM,253049,15,Lonza,1.0,NaN,NaN
4,Dec 13 2022 4:16PM,253040,10,TherapeuticsMD Elli Sample,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 4:58PM,253053,15,"Brookfield Pharmaceuticals, LLC",25.0,0.0,0.0
1,Dec 13 2022 4:57PM,253051,15,"Alliance Pharma, Inc.",15.0,0.0,0.0
2,Dec 13 2022 4:56PM,253052,10,"TherapeuticsMD, Inc.",1.0,0.0,0.0
3,Dec 13 2022 4:40PM,253049,15,Lonza,1.0,0.0,0.0
4,Dec 13 2022 4:16PM,253040,10,TherapeuticsMD Elli Sample,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5566063,77.0,19.0,6.0
12,252958,1.0,0.0,0.0
15,759153,41.0,0.0,0.0
16,253035,1.0,0.0,0.0
17,252964,1.0,0.0,0.0
19,759057,3.0,0.0,0.0
20,505947,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5566063,77.0,19.0,6.0
1,12,252958,1.0,0.0,0.0
2,15,759153,41.0,0.0,0.0
3,16,253035,1.0,0.0,0.0
4,17,252964,1.0,0.0,0.0
5,19,759057,3.0,0.0,0.0
6,20,505947,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77.0,19.0,6.0
1,12,1.0,0.0,0.0
2,15,41.0,0.0,0.0
3,16,1.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,3.0,0.0,0.0
6,20,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,12,Released,1.0
2,15,Released,41.0
3,16,Released,1.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20
Status,,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,19.0,0.0,0.0,0.0,0.0,0.0,0.0
Released,77.0,1.0,41.0,1.0,1.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Released,77.0,1.0,41.0,1.0,1.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Released,77.0,1.0,41.0,1.0,1.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()